“Online” es un conjunto de que contiene todas las transacciones que ocurrieron entre el 01/12/2010 y el 09/12/2011 para un comercio minorista en línea (Reino Unido). La empresa vende principalmente regalos únicos para todas las ocasiones. Muchos clientes de la empresa son mayoristas.  El objetivo es realizar un agrupamiento de los clientes y elegir a qué grupos dirigirse.
Vamos a analizar los Clientes en función de los siguientes factores: 

* Días: número de días desde la última compra 
* Frecuencia: número de transacciones 
* Gasto: gastos totales 


 1. Cargamos las librerias y los datos

In [1]:
# Importamos librerias para manejo de los datos

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import datetime as dt


# Import librerias para análisis de cluster
import sklearn
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from scipy.cluster.hierarchy import linkage
from scipy.cluster.hierarchy import dendrogram
from scipy.cluster.hierarchy import cut_tree

In [8]:
import xlrd

In [10]:
# Lectura de los datos

retail = pd.read_csv('Online.xlsx - Hoja1.csv')
retail.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/10 8:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,12/1/10 8:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/10 8:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/10 8:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/10 8:26,3.39,17850.0,United Kingdom


In [11]:
# Forma del objeto
retail.shape

(541909, 8)

In [12]:
#Análisis descriptivo
retail.describe() 

,Quantity,UnitPrice,CustomerID
count,541909.000000,541909.000000,406829.000000
mean,9.552250,4.611114,15287.690570
std,218.081158,96.759853,1713.600303
min,-80995.000000,-11062.060000,12346.000000
25%,1.000000,1.250000,13953.000000
50%,3.000000,2.080000,15152.000000
75%,10.000000,4.130000,16791.000000
max,80995.000000,38970.000000,18287.000000


2. Limpieza de los datos


In [13]:
#Eliminamos filas con valores perdidos
retail = retail.dropna()
retail.shape

(406829, 8)

In [14]:
# Creamos la variable Gasto

retail['Gasto'] = retail['Quantity']*retail['UnitPrice']
rfm_m = retail.groupby('CustomerID')['Gasto'].sum()
rfm_m = rfm_m.reset_index()
rfm_m.head()

,CustomerID,Gasto
0,12346.0,0.00
1,12347.0,4310.00
2,12348.0,1797.24
3,12349.0,1757.55
4,12350.0,334.40


In [15]:
#Creamos la variable Frecuencia
rfm_f = retail.groupby('CustomerID')['InvoiceNo'].count()
rfm_f = rfm_f.reset_index()
rfm_f.columns = ['CustomerID', 'Frequencia']
rfm_f.head()

,CustomerID,Frequencia
0,12346.0,2
1,12347.0,182
2,12348.0,31
3,12349.0,73
4,12350.0,17


In [16]:
#Unimos rfm_m y rfm_f
rfm = pd.merge(rfm_m, rfm_f, on='CustomerID', how='inner')
rfm.head()

,CustomerID,Gasto,Frequencia
0,12346.0,0.00,2
1,12347.0,4310.00,182
2,12348.0,1797.24,31
3,12349.0,1757.55,73
4,12350.0,334.40,17


In [18]:
# Creamos la variable Dias. 
# Convertimos la variable fecha al tipo de datos apropiados

retail['InvoiceDate'] = pd.to_datetime(retail['InvoiceDate'],format='%d-%m-%Y %H:%M')

ValueError: time data '12/1/10 8:26' does not match format '%d-%m-%Y %H:%M' (match)

In [ ]:
#Calculamos la fecha máxima para conocer la fecha de la última transacción
max_date = max(retail['InvoiceDate'])
max_date

In [ ]:
#Calculamos la diferencia entre la fecha máxima y la fecha de la transacción
retail['Dias'] = max_date - retail['InvoiceDate']
retail.head()

In [ ]:
#Calculamos la fecha de la última transacción para obtener la antigüedad de los clientes
rfm_p = retail.groupby('CustomerID')['Dias'].min()
rfm_p = rfm_p.reset_index()
rfm_p.head()

In [ ]:
#Extraemos el número de días
rfm_p['Dias'] = rfm_p['Dias'].dt.days
rfm_p.head()

In [ ]:
#Unimos para obtener la base final
rfm = pd.merge(rfm, rfm_p, on='CustomerID', how='inner')
rfm.columns = ['CustomerID', 'Gasto', 'Frequencia', 'Dias']
rfm.head()

In [ ]:
# Eliminamos los valores atípicos de Gasto
Q1 = rfm.Gasto.quantile(0.05)
Q3 = rfm.Gasto.quantile(0.95)
IQR = Q3 - Q1
rfm = rfm[(rfm.Gasto >= Q1 - 1.5*IQR) & (rfm.Gasto <= Q3 + 1.5*IQR)]

# Eliminamos los valores atípicos de Dias
Q1 = rfm.Dias.quantile(0.05)
Q3 = rfm.Dias.quantile(0.95)
IQR = Q3 - Q1
rfm = rfm[(rfm.Dias >= Q1 - 1.5*IQR) & (rfm.Dias <= Q3 + 1.5*IQR)]

# Eliminamos los valores atípicos de la Frecuencia
Q1 = rfm.Frequencia.quantile(0.05)
Q3 = rfm.Frequencia.quantile(0.95)
IQR = Q3 - Q1
rfm = rfm[(rfm.Frequencia >= Q1 - 1.5*IQR) & (rfm.Frequencia <= Q3 + 1.5*IQR)]

In [ ]:
#Estandarizamos los valores

rfm_df = rfm[['Gasto', 'Frequencia', 'Dias']]

scaler = StandardScaler()

rfm_df_scaled = scaler.fit_transform(rfm_df)
rfm_df_scaled.shape

In [ ]:
rfm_df_scaled = pd.DataFrame(rfm_df_scaled)
rfm_df_scaled.columns = ['Gasto', 'Frequencia', 'Dias']
rfm_df_scaled.head()

In [ ]:
#Método de Ward
mergings = linkage(rfm_df_scaled, method="ward", metric='euclidean')
dendrogram(mergings)
plt.show()

In [ ]:
# Elbow

ssd = []
range_n_clusters = [2, 3, 4, 5, 6, 7, 8]
for num_clusters in range_n_clusters:
    kmeans = KMeans(n_clusters=num_clusters, max_iter=50)
    kmeans.fit(rfm_df_scaled)
    
    ssd.append(kmeans.inertia_)
    
# Gráfico
plt.plot(ssd)

In [ ]:
# Cálculo de Si
range_n_clusters = [2, 3, 4, 5, 6, 7, 8]

for num_clusters in range_n_clusters:
    
    # intialise kmeans
    kmeans = KMeans(n_clusters=num_clusters, max_iter=50)
    kmeans.fit(rfm_df_scaled)
    
    cluster_labels = kmeans.labels_
    
    # silhouette score
    silhouette_avg = silhouette_score(rfm_df_scaled, cluster_labels)
    print("For n_clusters={0}, the silhouette score is {1}".format(num_clusters, silhouette_avg))

In [ ]:
# Modelo final k=3
kmeans = KMeans(n_clusters=3, max_iter=50)
kmeans.fit(rfm_df_scaled)

In [ ]:
rfm['Cluster_Id'] = kmeans.labels_
rfm.head()

In [ ]:
# Box plot Gasto

sns.boxplot(x='Cluster_Id', y='Gasto', data=rfm)

In [ ]:
# Box plot Cluster Frequencia

sns.boxplot(x='Cluster_Id', y='Frequencia', data=rfm)

In [ ]:
# Box plot Días
sns.boxplot(x='Cluster_Id', y='Dias', data=rfm)

Los clientes del Cluster 1 son los clientes con una gran cantidad de transacciones en comparación con otros clientes. Su gasto es superior al de otros clientes ya que son compradores frecuentes. 
Los clientes del Cluster 2 no son compradores frecuentes y su última compra fue hace más tiempo, por lo tanto, son los menos importantes desde el punto de vista comercial. 
Los clientes del Cluster 0 realizaron su compra más recientemente pero tienen poca frecuencia de compra.


Como actividad puede realizar los dendogramas con otros métodos de aglomeración. También ajuste k means con 2 grupos y analice.
